In [7]:
from Custom_Tetris_Env.tetris import TetrisEnv
from Custom_Tetris_Env.jp import JoypadSpace
from Custom_Tetris_Env.actions import MOVEMENT
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import CheckpointCallback


checkpoint_callback = CheckpointCallback(save_freq=100000, save_path='./logs/',
                                         name_prefix='dqn_model')

env = TetrisEnv()
env = JoypadSpace(env, MOVEMENT)

model = DQN("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [8]:
# Train the agent
model.learn(total_timesteps=int(1e6), callback=checkpoint_callback)

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.17e+03 |
|    ep_rew_mean      | -20      |
|    exploration rate | 0.689    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 653      |
|    time_elapsed     | 50       |
|    total timesteps  | 32686    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.29e+03 |
|    ep_rew_mean      | -20      |
|    exploration rate | 0.37     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 631      |
|    time_elapsed     | 104      |
|    total timesteps  | 66300    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000455 |
|    n_updates        | 4074     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean    

KeyboardInterrupt: 

: 

In [7]:
# save model to file
model.save("save_models/ppo_tetris")

In [3]:
done = False
obs = env.reset()


while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()

TypeError: unhashable type: 'numpy.ndarray'